# Tutorial 5: Creating Connections Between Nodes

## Learning Objectives
- Create network links between devices
- Build end-to-end network topology
- Establish ROADM-to-ROADM connections
- Create transponder-to-ROADM links
- Provision end-to-end lightpath services

## Prerequisites
- Completed all previous tutorials (1-4)
- Mounted devices with created interfaces

## Overview
Network connections are the foundation of optical transport networks. This tutorial demonstrates creating links and provisioning services.

## Step 1: Student Configuration

Configure your student ID to access the correct isolated environment.

In [ ]:
# === STUDENT CONFIGURATION ===
STUDENT_ID = "1"  # Change this to your assigned student ID

# Generated configuration
NETWORK_NAME = f"student{STUDENT_ID}-network"
TPCE_CONTAINER_NAME = f"student{STUDENT_ID}-tpce"

print(f"Student ID: {STUDENT_ID}")
print(f"Network: {NETWORK_NAME}")
print(f"TransportPCE Container: {TPCE_CONTAINER_NAME}")

## Step 2: Import Required Libraries

Import libraries for network topology management and service creation.

In [ ]:
# Import libraries
import docker
import requests
import json
import time
from requests.auth import HTTPBasicAuth
from common_utils import StatusIndicators, print_status, DockerHelper

# Initialize Docker helper
docker_helper = DockerHelper()

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

print("Libraries imported successfully")

## Step 3: Define Helper Functions

Create functions for topology management and service provisioning operations.

In [ ]:
# Helper functions

def get_openroadm_topology(tpce_ip):
    """Get current OpenROADM network topology"""
    url = f"http://{tpce_ip}:8181/rests/data/ietf-network:networks/network=openroadm-topology"
    headers = {'Accept': 'application/json'}
    auth = HTTPBasicAuth('admin', 'admin')
    
    print(f"\nGetting OpenROADM topology...")
    print(f"URL: {url}")
    print(f"Method: GET")

    try:
        response = requests.get(url, headers=headers, auth=auth, timeout=10)
        print(f"Response Status: {response.status_code}")

        if response.status_code == 200:
            data = response.json()
            if 'ietf-network:network' in data:
                return data['ietf-network:network'][0]
    except Exception as e:
        print(f"Error getting topology: {e}")
    return None

def create_roadm_xpdr_links(tpce_ip, roadm_name, xpdr_name):
    """Create bidirectional links between ROADM and XPDR"""

    payload = {
        "input": {
            "links-input": {
                "xpdr-node": xpdr_name,
                "xpdr-num": "1",
                "network-num": "1",
                "rdm-node": roadm_name,
                "srg-num": "1",
                "termination-point-num": "SRG1-PP1-TXRX"
            }
        }
    }

    print(f"\nCreating ROADM-XPDR links...")
    print(f"Payload: {json.dumps(payload, indent=2)}")

    headers = {'Content-Type': 'application/json'}
    auth = HTTPBasicAuth('admin', 'admin')

    # Create ROADM to XPDR link
    url1 = f"http://{tpce_ip}:8181/rests/operations/transportpce-networkutils:init-rdm-xpdr-links"
    print(f"\n--- Request 1: ROADM → XPDR Link ---")
    print(f"URL: {url1}")
    print(f"Method: POST")

    response1 = requests.post(url1, json=payload, headers=headers, auth=auth, timeout=30)
    print(f"Response Status: {response1.status_code}")
    print(f"Response Body: {response1.text if response1.text else '[Empty]'}")

    # Create XPDR to ROADM link
    url2 = f"http://{tpce_ip}:8181/rests/operations/transportpce-networkutils:init-xpdr-rdm-links"
    print(f"\n--- Request 2: XPDR → ROADM Link ---")
    print(f"URL: {url2}")
    print(f"Method: POST")

    response2 = requests.post(url2, json=payload, headers=headers, auth=auth, timeout=30)
    print(f"Response Status: {response2.status_code}")
    print(f"Response Body: {response2.text if response2.text else '[Empty]'}")

    return response1.status_code == 200 and response2.status_code == 200

def create_service(tpce_ip, service_name, source_node, dest_node):
    """Create end-to-end optical service"""

    payload = {
        "input": {
            "service-name": service_name,
            "connection-type": "service",
            "service-a-end": {
                "service-rate": 400,
                "node-id": source_node,
                "service-format": "Ethernet"
            },
            "service-z-end": {
                "service-rate": 400,
                "node-id": dest_node,
                "service-format": "Ethernet"
            }
        }
    }

    url = f"http://{tpce_ip}:8181/rests/operations/org-openroadm-service:service-create"

    print(f"\n--- Creating Service ---")
    print(f"Service Name: {service_name}")
    print(f"URL: {url}")
    print(f"Method: POST")
    print(f"\nPayload:")
    print(json.dumps(payload, indent=2))

    headers = {'Content-Type': 'application/json'}
    auth = HTTPBasicAuth('admin', 'admin')

    response = requests.post(url, json=payload, headers=headers, auth=auth, timeout=30)

    print(f"\nResponse Status: {response.status_code}")
    print(f"Response Body: {response.text}")
    
    return response

print("Helper functions defined")

## Step 4: Initialize Connection Manager

Establish connection to TransportPCE for network operations.

In [ ]:
# Initialize connection manager
tpce_ip = docker_helper.get_container_ip(TPCE_CONTAINER_NAME, NETWORK_NAME)

if tpce_ip:
    print(f"TransportPCE IP: {tpce_ip}")
    print("Connection manager initialized")
else:
    print("Could not connect to TransportPCE")

## Step 5: Examine Network Topology

Analyze the current state of the network topology and available devices.

In [ ]:
# Examine current network topology
if tpce_ip:
    print("=== EXAMINING NETWORK TOPOLOGY ===")
    print("Understanding the current state before creating connections...")
    
    topology = get_openroadm_topology(tpce_ip)
    
    if topology:
        nodes = topology.get('node', [])
        links = topology.get('ietf-network-topology:link', [])
        
        print(f"\nNetwork Overview:")
        print(f"  Total Nodes: {len(nodes)}")
        print(f"  Existing Links: {len(links)}")
        
        if nodes:
            print(f"\nNodes discovered in topology:")
            roadm_count = 0
            xpdr_count = 0
            
            for node in nodes:
                node_id = node.get('node-id', 'Unknown')
                node_type = node.get('org-openroadm-common-network:node-type', 'Unknown')
                print(f"  - {node_id} (Type: {node_type})")
                
                # Show termination points if available
                tps = node.get('ietf-network-topology:termination-point', [])
                if tps:
                    tp_names = [tp.get('tp-id') for tp in tps[:3]]
                    print(f"    Termination points: {tp_names}")
                
                if node_type == 'ROADM':
                    roadm_count += 1
                elif node_type == 'XPONDER':
                    xpdr_count += 1
            
            print(f"\nDevice Summary:")
            print(f"  ROADM devices: {roadm_count}")
            print(f"  Transponder devices: {xpdr_count}")
            
            if xpdr_count == 0:
                print(f"\n⚠ WARNING: No transponder (XPONDER) found in topology!")

        
        if links:
            print(f"\nExisting network links:")
            for i, link in enumerate(links, 1):
                link_id = link.get('link-id', 'Unknown')
                source = link.get('source', {})
                dest = link.get('destination', {})
                print(f"  {i}. {source.get('source-node', 'Unknown')} → {dest.get('dest-node', 'Unknown')}")
                print(f"     Link ID: {link_id}")
        else:
            print(f"\nNo existing links found")
            print("This is normal - services will create links automatically")
    else:
        print("Could not retrieve network topology")
        print("Ensure devices are mounted (Tutorial 3 completed)")
else:
    print("Cannot examine topology - no TransportPCE connection")

## Step 6: Create ROADM-XPDR Links

Create bidirectional links between ROADM and transponder devices using TransportPCE network utilities.

In [ ]:
# Create ROADM-XPDR Links
if tpce_ip:
    print("=== CREATING ROADM-XPDR LINKS ===")
    
    # Discover devices in student network
    try:
        network = docker_helper.client.networks.get(NETWORK_NAME)
        network_info = docker_helper.client.api.inspect_network(network.id)
        containers = network_info.get('Containers', {})
        
        roadm_device = None
        xpdr_device = None
        
        for container_info in containers.values():
            name = container_info.get('Name', '')
            if 'roadm' in name.lower():
                # Extract device name (e.g., "roadm-a" from "student1-roadm-a")
                roadm_device = name.replace(f'student{STUDENT_ID}-', '')
            elif 'tpdr' in name.lower():
                # Extract device name (e.g., "tpdr-a" from "student1-tpdr-a")
                xpdr_device = name.replace(f'student{STUDENT_ID}-', '')
        
        if roadm_device and xpdr_device:
            print(f"\nDiscovered devices in student {STUDENT_ID} network:")
            print(f"  ROADM: {roadm_device}")
            print(f"  XPDR: {xpdr_device}")
            
            # Check if devices are mounted in TransportPCE
            netconf_url = f"http://{tpce_ip}:8181/rests/data/network-topology:network-topology/topology=topology-netconf"
            headers = {'Accept': 'application/json'}
            auth = HTTPBasicAuth('admin', 'admin')
            
            response = requests.get(netconf_url, headers=headers, auth=auth, timeout=10)
            print(f"\nChecking mounted devices...")
            
            if response.status_code == 200:
                data = response.json()
                topology = data.get('network-topology:topology', [{}])[0]
                nodes = topology.get('node', [])
                mounted_devices = [node.get('node-id') for node in nodes]
                
                if roadm_device in mounted_devices and xpdr_device in mounted_devices:
                    print(f"✓ Both devices are mounted in TransportPCE")
                    
                    # Get topology before
                    topology_before = get_openroadm_topology(tpce_ip)
                    links_before = []
                    if topology_before:
                        links_before = topology_before.get('ietf-network-topology:link', [])
                        print(f"\nCurrent links: {len(links_before)}")
                    
                    # Create the links
                    success = create_roadm_xpdr_links(tpce_ip, roadm_device, xpdr_device)
                    
                    if success:
                        print(f"\n✓ Link creation successful")
                        
                        # Wait and check topology
                        print(f"Waiting 3 seconds for topology update...")
                        time.sleep(3)
                        
                        topology_after = get_openroadm_topology(tpce_ip)
                        if topology_after:
                            links_after = topology_after.get('ietf-network-topology:link', [])
                            print(f"Links after: {len(links_after)}")
                            new_links = len(links_after) - len(links_before)
                            if new_links > 0:
                                print(f"✓ {new_links} new links created")
                            else:
                                print(f"⚠ No new links detected")
                    else:
                        print(f"\n✗ Link creation failed")
                else:
                    print(f"\n✗ Devices not mounted in TransportPCE:")
                    if roadm_device not in mounted_devices:
                        print(f"  Missing: {roadm_device}")
                    if xpdr_device not in mounted_devices:
                        print(f"  Missing: {xpdr_device}")
                    print(f"  Please complete Tutorial 3 first")
        else:
            print(f"\n✗ Could not find devices in student {STUDENT_ID} network")
            if not roadm_device:
                print(f"  Missing ROADM device")
            if not xpdr_device:
                print(f"  Missing XPDR device")
            print(f"  Please complete Tutorials 1-2 first")
            
    except Exception as e:
        print(f"Error discovering devices: {e}")
else:
    print("Cannot create links - no TransportPCE connection")

## Step 7: Verify Network Updates

Check the updated topology after link creation to confirm connections and analyze the network changes.

In [ ]:
# Verify network topology
if tpce_ip:
    print("=== VERIFYING NETWORK TOPOLOGY ===")
    
    # Get current topology
    topology = get_openroadm_topology(tpce_ip)
    
    if topology:
        nodes = topology.get('node', [])
        links = topology.get('ietf-network-topology:link', [])
        
        print(f"Network Status:")
        print(f"  Total Nodes: {len(nodes)}")
        print(f"  Total Links: {len(links)}")
        
        # Count node types
        node_types = {}
        for node in nodes:
            node_type = node.get('org-openroadm-common-network:node-type', 'Unknown')
            node_types[node_type] = node_types.get(node_type, 0) + 1
        
        print(f"\nNode Types:")
        for node_type, count in node_types.items():
            print(f"  {node_type}: {count}")
        
        # Show first few links
        if links:
            print(f"\nShowing first 5 links:")
            for i, link in enumerate(links):
                source = link.get('source', {})
                dest = link.get('destination', {})
                print(f"  {i}. {source.get('source-node', 'Unknown')} → {dest.get('dest-node', 'Unknown')}")

    else:
        print("Could not retrieve topology")
else:
    print("Cannot verify topology - no TransportPCE connection")